<div style="text-align:left; background-color:gray; padding:0px;">
  <h1 style="color:white;">Création d'un modèle simple</h1>
</div>
<br>

#### Suite à l'EDA (Analyse exploratoire de données) réalisé précédemment, nous allons créer dans ce notebook, un modèle simple de régression linéaire en utilisant l'algorithme d'apprentissage automatique RandomForestRegressor. La même préparation de données sera effectuée et on conservera uniquement les variables SURFACE_BATi et NB_PIECES. Le but est de mettre en place le monitoring avec MLFlow et de pouvoir livrer la première version de l'application au plus vite. Les performances de ce premier modèle ne seront donc, dans un premier temps, pas très bonne.
#### Par la suite, dans d'autres fichiers python nous pourrons améliorer les prédictions en changeant d'algorithme d'apprentissage ou, en créant un modèle par région et en ajoutant des variables comme par exemple le type de bien, une information sur la zone de prix (référence aux cartes de baromètre des prix réalisé avec folium), un coefficient permettant d'ajuster le prix dans le temps...  

<div style="text-align:left; background-color:gray; padding:0px;">
  <h1 style="color:white;">Import des librairies</h1>
</div>
<br>

In [62]:
# Visualisation de données
import pandas as pd
pd.set_option('display.max_columns', None)
import seaborn as sns
sns.set_context("talk")
sns.set_style("darkgrid")

<div style="text-align:left; background-color:gray; padding:0px;">
  <h1 style="color:white;">Récupération et nettoyage des données</h1></div><br>

In [63]:
# Récupération des données
df = pd.read_csv("datas_rds.csv", low_memory=False)
# Sélection des données non nulles
df = df.loc[(df.MONTANT>0) & (df.NB_PIECES>0) & (df.SURFACE_BATI>0),:]
# Suppression des valeurs extremes en région Bretagne
df = df[~((df.Name_region=="Bretagne")&(df.MONTANT>6.5e6))]
# Sélection des données
df = df.loc[:,["SURFACE_BATI","NB_PIECES","NAME_TYPE_BIEN","Name_region","MONTANT"]]
# Suppression des lignes dupliquées
df = df.drop_duplicates()
df.head()

,SURFACE_BATI,NB_PIECES,NAME_TYPE_BIEN,Name_region,MONTANT
0,233,6,Maison,Auvergne-Rhône-Alpes,450000
1,140,5,Maison,Auvergne-Rhône-Alpes,270600
2,88,4,Maison,Auvergne-Rhône-Alpes,172000
3,81,3,Appartement,Auvergne-Rhône-Alpes,131500
4,159,5,Maison,Auvergne-Rhône-Alpes,247000


#### Étant donné que nous créons un modèle unique, qu'au sein d'une même région il peut y avoir une grosse différence de prix entre deux bien d'une même surface (exemple entre la côte et les terres) et que nous ne pouvons pas ajouter la commune dans l'entraînement du modèle, nous allons donc supprimer les outliers(valeurs extremes) afin d'éviter le sur-ajustement.

In [64]:
# Suppression des outliers afin de moins perturber le modèle lors de l'apprentissage
def filtrer_outliers(groupe):
    Q1 = groupe['MONTANT'].quantile(0.25)
    Q3 = groupe['MONTANT'].quantile(0.75)
    IQR = Q3 - Q1 # Range interquartile
    # Convention sur la statistique de l'IQR pour determiner les outliers
    borne_inf = Q1 - 1.5 * IQR
    borne_sup = Q3 + 1.5 * IQR
    return groupe[(groupe['MONTANT'] >= borne_inf) & (groupe['MONTANT'] <= borne_sup)]

df = df.groupby('Name_region').apply(filtrer_outliers)
# Rest de L'index
df = df.reset_index(drop=True)
df.head()

,SURFACE_BATI,NB_PIECES,NAME_TYPE_BIEN,Name_region,MONTANT
0,233,6,Maison,Auvergne-Rhône-Alpes,450000
1,140,5,Maison,Auvergne-Rhône-Alpes,270600
2,88,4,Maison,Auvergne-Rhône-Alpes,172000
3,81,3,Appartement,Auvergne-Rhône-Alpes,131500
4,159,5,Maison,Auvergne-Rhône-Alpes,247000


In [65]:
print(f"Il reste {df.shape[0]} ventes pour réaliser le modèle")

Il reste 2421765 ventes pour réaliser le modèle


<div style="text-align:left; background-color:gray; padding:0px;">
  <h1 style="color:white;">Labellisation et Standardisation des données</h1>
</div><br>

In [66]:
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder

def encod_scal(df : pd.DataFrame) -> (pd.DataFrame , dict, dict):
    """ 
    Fonction permettant de labelliser puis de standardiser un Data frame  

    Args :
    - df (pd.DataFrame) : Les données à labelliser puis standardiser

    Return :
    - df (pd.DataFrame) : Les données labellisées et standardisées
    - encoders (dict) : Dictionnaire stockant les encodeurs pour chaque variable catégorielle
    - scalers (dict) : Dictionnaire stockant les scalers pour chaque variable numérique
    - non_numerical (list) : Liste des variables catégorielle
    - features (list) : Liste des colonnes à standardiser
    """
    # Sélection des variables non numériques
    non_numerical = df.select_dtypes(exclude=['number']).columns.to_list()
    # Sélection des colonnes à traiter (toutes sauf la valeur à prédire)
    features = df.drop('MONTANT', axis=1).columns

    # Dictionnaire où seront stockés les LabelEncoder et Scaler afin
    # de pouvoir inverser la labellisation et la standardisation
    encoders = {}
    scalers = {}

    # Encodage des variables catégorielles
    for col in non_numerical:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        encoders[col] = le
    # Normalisation des données
    for col in features:
        scaler = StandardScaler()
        df[col] = scaler.fit_transform(df[col].values.reshape(-1, 1))
        scalers[col] = scaler
    return df,encoders,scalers, non_numerical, features

def reverse_scal_encod(df : pd.DataFrame, encoders : dict, scalers : dict,
                       non_numerical: list, features: list) -> pd.DataFrame:
    """ 
    Fonction permettant d'inverser la standardisation puis la labellisation  

    Args:
    - df (pd.DataFrame) : Data frame qui a été labellisé et standardisé  
    - encoders (dict) : Dictionnaire contenant les encodeurs pour chaque variable catégorielle
    - scalers (dict) : Dictionnaire contenant les scalers pour chaque variable numérique
    - non_numerical (list) : Liste des variables catégorielle
    - features (list) : Liste des colonnes à standardiser
    
    Returns:
    - df (pd.DataFrame) : Data frame avec les valeurs d'origine
    """
    # Inversion de la standardisation des données
    for col in features:
        scaler = scalers[col]
        df[col] = scaler.inverse_transform(df[col].values.reshape(-1, 1))

    # Inversion de l'encodage des variables catégorielles
    for col in non_numerical:
        le = encoders[col]
        df[col] = le.inverse_transform(df[col].astype(int))
    return df

# Labellisation et standardisation
df ,encoders,scalers,non_numerical,features = encod_scal(df) 

<div style="text-align:left; background-color:gray; padding:0px;">
  <h1 style="color:white;">Séparation des données en un jeu d'entraînement et un jeu de test</h1>
</div><br>

In [67]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1],df.iloc[:,-1],test_size=0.8,random_state=42)

<div style="text-align:left; background-color:gray; padding:0px;">
  <h1 style="color:white;">Entraînement sans MLFlow et sans recherche d'hyperparamètres</h1>
</div><br>

In [68]:
# Entraînement du modèle
from sklearn.ensemble import RandomForestRegressor

model=RandomForestRegressor()
model.fit(X_train,y_train)

RandomForestRegressor()

In [69]:
# Performance du modèle
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np 

y_pred = model.predict(X_test)
def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = calculate_mape(y_test, y_pred)


# Affichage des résultats
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Écart de prix moyen : {rmse:.2f}")
print(f"R-squared (R2): {r2:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

Mean Squared Error (MSE): 12979813609.39
Écart de prix moyen : 113928.98
R-squared (R2): 0.36
Mean Absolute Error (MAE): 85330.15
Mean Absolute Percentage Error (MAPE): 14941.65%


#### La moyenne des erreurs entre prédictions et réalité et de 85 329€

#### Visualisation du bien avec le plus grand écart :

In [70]:
ecarts = np.abs(y_pred - y_test)
index_ecart_maximum = np.argmax(ecarts)


observation = X_test.iloc[index_ecart_maximum,:].to_frame().transpose()
print(f"Observation N°{index_ecart_maximum} :",
          "valeur prédite :", y_pred[index_ecart_maximum],
          "valeur réelle : ",y_test.iloc[index_ecart_maximum],
          "Différence : ", y_test.iloc[index_ecart_maximum]-y_pred[index_ecart_maximum])

observation = reverse_scal_encod(observation,encoders,scalers,non_numerical,features)
print(observation)


Observation N°144509 : valeur prédite : 757181.5778214284 valeur réelle :  4100 Différence :  -753081.5778214284
         SURFACE_BATI  NB_PIECES NAME_TYPE_BIEN    Name_region
2329684         139.0        4.0    Appartement  Île-de-France


##### Ici nous pouvons voir qu'il y a encore des valeurs aberrantes, un appartement de 183m² en Île de France est à mon avis impossible !!! 
=> Regardons le nombre de biens inférieurs à 10 000€ :

In [71]:
le = encoders["Name_region"]
scaler = scalers["Name_region"]
detect_100000 = df.loc[df.MONTANT < 10000, :].copy()

# Inversion de la standardisation des données
detect_100000["Name_region"] = scaler.inverse_transform(detect_100000["Name_region"].values.reshape(-1, 1))

# Inversion de l'encodage des variables catégorielles
detect_100000["Name_region"] = le.inverse_transform(detect_100000["Name_region"].astype(int))

detect_100000["Name_region"].value_counts()


Name_region
Nouvelle-Aquitaine            2319
Auvergne-Rhône-Alpes          2292
Occitanie                     1910
Bourgogne-Franche-Comté       1549
Île-de-France                 1353
Grand Est                     1332
Pays de la Loire              1119
Bretagne                      1094
Centre-Val de Loire           1024
Provence-Alpes-Côte d'Azur     943
Hauts-de-France                941
Normandie                      739
Guadeloupe                     319
Martinique                     313
Corse                          199
La Réunion                     199
Guyane                         126
Name: count, dtype: int64

### Conclusion : Après une petite recherche sur le site Leboncoin nous pouvons remarqué que les biens vendus en dessous de 10 000€ sont parfois des caves, places de parking ou des biens acquis en temps partagé(exemple appartement à la montagne). La suppression des outliers n'a à priori pas permis d'enlever ces biens dans l'entraînement du modèle ce qui génère une baisse des performances.

<div style="text-align:left; background-color:gray; padding:0px;">
  <h1 style="color:white;">Entraînement et résultats en enlevant les biens de moins de 10 000€</h1>
</div><br>

In [72]:
# Récupération des données
df = pd.read_csv("datas_rds.csv", low_memory=False)
# Sélection des données non nulles
df = df.loc[(df.MONTANT>10000) & (df.NB_PIECES>0) & (df.SURFACE_BATI>0),:]
# Suppression des valeurs extremes en région Bretagne
df = df[~((df.Name_region=="Bretagne")&(df.MONTANT>6.5e6))]
# Sélection des données
df = df.loc[:,["SURFACE_BATI","NB_PIECES","NAME_TYPE_BIEN","Name_region","MONTANT"]]
# Suppression des lignes dupliquées
df = df.drop_duplicates()
# Suppression des outliers afin de moins perturber le modèle lors de l'apprentissage
def filtrer_outliers(groupe):
    Q1 = groupe['MONTANT'].quantile(0.25)
    Q3 = groupe['MONTANT'].quantile(0.75)
    IQR = Q3 - Q1 # Range interquartile
    # Convention sur la statistique de l'IQR pour determiner les outliers
    borne_inf = Q1 - 1.5 * IQR
    borne_sup = Q3 + 1.5 * IQR
    return groupe[(groupe['MONTANT'] >= borne_inf) & (groupe['MONTANT'] <= borne_sup)]

df = df.groupby('Name_region').apply(filtrer_outliers)
# Rest de L'index
df = df.reset_index(drop=True)
# Labellisation et standardisation
df, encoders, scalers, non_numerical, features = encod_scal(df) 
# Split de données
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1],df.iloc[:,-1],test_size=0.8,random_state=42)
# Entraînement des données
model=RandomForestRegressor()
model.fit(X_train,y_train)
# Prédiction et métriques
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = calculate_mape(y_test, y_pred)


# Affichage des résultats
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Écart de prix moyen : {rmse:.2f}")
print(f"R-squared (R2): {r2:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")


Mean Squared Error (MSE): 12788971561.09
Écart de prix moyen : 113088.34
R-squared (R2): 0.36
Mean Absolute Error (MAE): 84814.47
Mean Absolute Percentage Error (MAPE): 67.69%


In [73]:
print("COMPARAISON DES 5 PREMIERS BIENS /\n")
for i in range(5):
    observation = X_test.iloc[i,:].to_frame().transpose()
    print(f"Observation N°{i} :",
          "valeur prédite :", round(model.predict(observation)[0],2),
          "valeur réelle : ",y_test.iloc[i])
    
    observation = reverse_scal_encod(observation,encoders,scalers,non_numerical,features)
    print(observation)
    print("------------------------------------------------------------")


COMPARAISON DES 5 PREMIERS BIENS /

Observation N°0 : valeur prédite : 242459.74 valeur réelle :  144000
         SURFACE_BATI  NB_PIECES NAME_TYPE_BIEN         Name_region
1430383          73.0        3.0    Appartement  Nouvelle-Aquitaine
------------------------------------------------------------
Observation N°1 : valeur prédite : 108465.94 valeur réelle :  84000
         SURFACE_BATI  NB_PIECES NAME_TYPE_BIEN         Name_region
1260339          23.0        1.0    Appartement  Nouvelle-Aquitaine
------------------------------------------------------------
Observation N°2 : valeur prédite : 116162.65 valeur réelle :  247100
         SURFACE_BATI  NB_PIECES NAME_TYPE_BIEN Name_region
1074465          41.0        2.0    Appartement   Normandie
------------------------------------------------------------
Observation N°3 : valeur prédite : 133827.82 valeur réelle :  38000
         SURFACE_BATI  NB_PIECES NAME_TYPE_BIEN Name_region
1485248          50.0        2.0    Appartement   Occit

#### Conclusion : Le R2 score n'a pas évolué et le MAE a légèrement diminué mais en ce qui concerne MAPE on a drastiquement diminué, passant de 15 000% à 64%

In [74]:
ecarts = np.abs(y_pred - y_test)
index_ecart_maximum = np.argmax(ecarts)


observation = X_test.iloc[index_ecart_maximum,:].to_frame().transpose()
print(f"Observation N°{index_ecart_maximum} :",
          "valeur prédite :", y_pred[index_ecart_maximum],
          "valeur réelle : ",y_test.iloc[index_ecart_maximum],
          "Différence : ", y_test.iloc[index_ecart_maximum]-y_pred[index_ecart_maximum])

observation = reverse_scal_encod(observation,encoders,scalers,non_numerical,features)
print(observation)

Observation N°1799721 : valeur prédite : 761184.0644635743 valeur réelle :  11450 Différence :  -749734.0644635743
         SURFACE_BATI  NB_PIECES NAME_TYPE_BIEN    Name_region
2281147         148.0        6.0    Appartement  Île-de-France
